In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [71]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
meal_info = pd.read_csv('meal_info.csv')
center_info=pd.read_csv('fulfilment_center_info.csv')


In [72]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [73]:
df_test.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0


### So here we have to predict the num_orders for the weak 146 to 155 
First we will apredict without using the meal info and Center info dataset to compare the result.


In [74]:
df_train.describe()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
count,4.565480e+05,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.000000,456548.00000,456548.000000
mean,1.250096e+06,74.768771,82.105796,2024.337458,332.238933,354.156627,0.081152,0.10920,261.872760
std,1.443548e+05,41.524956,45.975046,547.420920,152.939723,160.715914,0.273069,0.31189,395.922798
min,1.000000e+06,1.000000,10.000000,1062.000000,2.970000,55.350000,0.000000,0.00000,13.000000
25%,1.124999e+06,39.000000,43.000000,1558.000000,228.950000,243.500000,0.000000,0.00000,54.000000
50%,1.250184e+06,76.000000,76.000000,1993.000000,296.820000,310.460000,0.000000,0.00000,136.000000
75%,1.375140e+06,111.000000,110.000000,2539.000000,445.230000,458.870000,0.000000,0.00000,324.000000
max,1.499999e+06,145.000000,186.000000,2956.000000,866.270000,866.270000,1.000000,1.00000,24299.000000


In [75]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 9 columns):
id                       456548 non-null int64
week                     456548 non-null int64
center_id                456548 non-null int64
meal_id                  456548 non-null int64
checkout_price           456548 non-null float64
base_price               456548 non-null float64
emailer_for_promotion    456548 non-null int64
homepage_featured        456548 non-null int64
num_orders               456548 non-null int64
dtypes: float64(2), int64(7)
memory usage: 31.3 MB


In [76]:
df_train.nunique()

id                       456548
week                        145
center_id                    77
meal_id                      51
checkout_price             1992
base_price                 1907
emailer_for_promotion         2
homepage_featured             2
num_orders                 1250
dtype: int64

## Approach 1:  
Since week, center_id and meal_id are categorical variable. And doing one hot encoding will make the dataset to large my first approach will be using decision tree.
For this first i will convert dtype of week, center_id and meal_id to object dtype and then apply the decision tree algorithm.

In [77]:
df_train1=df_train.copy()
df_test1=df_test.copy()

In [78]:
df_train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [79]:
df_train=df_train.astype({"meal_id" : object, "center_id":object})
df_test=df_test.astype({"meal_id" : object, "center_id":object})

In [80]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 9 columns):
id                       456548 non-null int64
week                     456548 non-null int64
center_id                456548 non-null object
meal_id                  456548 non-null object
checkout_price           456548 non-null float64
base_price               456548 non-null float64
emailer_for_promotion    456548 non-null int64
homepage_featured        456548 non-null int64
num_orders               456548 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 31.3+ MB


In [81]:
df_train=df_train.astype({"week":object})
df_test=df_test.astype({"week":object})

###  Now we will apply split the training set to train and test to evaluate it in the algorithm

In [82]:
X=df_train.iloc[:,1:-1].values
y=df_train.iloc[:,8].values

X_AV_test = df_test.iloc[:,1:].values

In [83]:
X[5]

array([1, 55, 1248, 251.23, 252.23, 0, 0], dtype=object)

In [84]:
y[5]

28

In [85]:
X.shape

(456548, 7)

In [86]:
y.shape

(456548,)

In [87]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Data columns (total 8 columns):
id                       32573 non-null int64
week                     32573 non-null object
center_id                32573 non-null object
meal_id                  32573 non-null object
checkout_price           32573 non-null float64
base_price               32573 non-null float64
emailer_for_promotion    32573 non-null int64
homepage_featured        32573 non-null int64
dtypes: float64(2), int64(3), object(3)
memory usage: 2.0+ MB


In [88]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

### Now implement the Decision tree regressor 

In [89]:
from sklearn.tree import DecisionTreeRegressor
DTR = DecisionTreeRegressor()
DTR.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [90]:
DTR.score(X_train,y_train)

1.0

In [91]:
y_pred=DTR.predict(X_test)

In [92]:
from sklearn.metrics import mean_squared_log_error

In [93]:
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.6730730426841289

###  Now we get a RMSLE of 0.67 , lets try it on the test data that anlytics vidhya provided or evaluation

In [94]:
y_AV_pred = DTR.predict(X_AV_test)

### Submission of result 

In [96]:
Submission = pd.DataFrame({'id': df_test['id'], 'num_orders' : y_AV_pred})

In [98]:
Submission.to_csv("C:\Machine Learning\Projects\Food_Demand_Forcasting\Submission1.csv", index=False)

#### After adding the submission on protal, i get a RMSLE of 0.81 which is very poor, so will try to devcrease the error. 
Approach we can try now
1. Tune the decision tree
2. Use Random forest and C